## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cv2
import matplotlib.pyplot as plt
import copy
import numpy as np

from src import model
from src import util
from src.body import Body
from src.hand import Hand

## Process Pose

load models

In [ ]:
body_estimation = Body('model/body_pose_model.pth')
hand_estimation = Hand('model/hand_pose_model.pth')

load image

In [ ]:
test_image = 'images/demo.jpg'

detect pose and display in image

In [ ]:
oriImg = cv2.imread(test_image)  # B,G,R order

# estimate body pose and draw
candidate, subset = body_estimation(oriImg)
canvas = copy.deepcopy(oriImg)
canvas = util.draw_bodypose(canvas, candidate, subset)

# estimate hand pose
hands_list = util.handDetect(candidate, subset, oriImg)

all_hand_peaks = []
for x, y, w, is_left in hands_list:
    # cv2.rectangle(canvas, (x, y), (x+w, y+w), (0, 255, 0), 2, lineType=cv2.LINE_AA)
    # cv2.putText(canvas, 'left' if is_left else 'right', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # if is_left:
        # plt.imshow(oriImg[y:y+w, x:x+w, :][:, :, [2, 1, 0]])
        # plt.show()
    peaks = hand_estimation(oriImg[y:y+w, x:x+w, :])
    peaks[:, 0] = np.where(peaks[:, 0]==0, peaks[:, 0], peaks[:, 0]+x)
    peaks[:, 1] = np.where(peaks[:, 1]==0, peaks[:, 1], peaks[:, 1]+y)
    # else:
    #     peaks = hand_estimation(cv2.flip(oriImg[y:y+w, x:x+w, :], 1))
    #     peaks[:, 0] = np.where(peaks[:, 0]==0, peaks[:, 0], w-peaks[:, 0]-1+x)
    #     peaks[:, 1] = np.where(peaks[:, 1]==0, peaks[:, 1], peaks[:, 1]+y)
    #     print(peaks)
    all_hand_peaks.append(peaks)

canvas = util.draw_handpose(canvas, all_hand_peaks)

plt.figure(figsize = (20,16))
plt.imshow(canvas[:, :, [2, 1, 0]])
plt.axis('off')
plt.show()

### Detect Pose in Video Frames

extract frames from video:

```
mkdir pose_front
ffmpeg -i pose_front.mov pose_front/%04d.jpg
mkdir pose_rear
ffmpeg -i pose_rear.mov pose_rear/%04d.jpg
```

In [ ]:
def detect(test_image):
    oriImg = None
    oriImg = cv2.imread(test_image)  # B,G,R order

    # estimate body pose and draw
    candidate, subset = body_estimation(oriImg)
    canvas = copy.deepcopy(oriImg)
    canvas = util.draw_bodypose(canvas, candidate, subset)

    # estimate hand pose
    hands_list = util.handDetect(candidate, subset, oriImg)

    all_hand_peaks = []
    for x, y, w, is_left in hands_list:
        # cv2.rectangle(canvas, (x, y), (x+w, y+w), (0, 255, 0), 2, lineType=cv2.LINE_AA)
        # cv2.putText(canvas, 'left' if is_left else 'right', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # if is_left:
            # plt.imshow(oriImg[y:y+w, x:x+w, :][:, :, [2, 1, 0]])
            # plt.show()
        peaks = hand_estimation(oriImg[y:y+w, x:x+w, :])
        peaks[:, 0] = np.where(peaks[:, 0]==0, peaks[:, 0], peaks[:, 0]+x)
        peaks[:, 1] = np.where(peaks[:, 1]==0, peaks[:, 1], peaks[:, 1]+y)
        # else:
        #     peaks = hand_estimation(cv2.flip(oriImg[y:y+w, x:x+w, :], 1))
        #     peaks[:, 0] = np.where(peaks[:, 0]==0, peaks[:, 0], w-peaks[:, 0]-1+x)
        #     peaks[:, 1] = np.where(peaks[:, 1]==0, peaks[:, 1], peaks[:, 1]+y)
        #     print(peaks)
        all_hand_peaks.append(peaks)

    canvas = util.draw_handpose(canvas, all_hand_peaks)

    # plt.figure(figsize = (20,16))
    # plt.imshow(canvas[:, :, [2, 1, 0]])
    # plt.axis('off')
    # plt.show()

    return canvas[:, :, [2, 1, 0]]

In [ ]:
result = detect('images/pose_front/0201.jpg')

plt.figure(figsize = (20,16))
plt.imshow(result)
plt.axis('off')
plt.show()

In [ ]:
# result

In [ ]:
cv2.imwrite("results/pose_front/0200.jpg", result)

loop over images and write to results

NOTE: I had to modify the body.py code in pytorch-openpose to correct a run error. See this thread https://github.com/Hzzone/pytorch-openpose/issues/1

In [ ]:
import glob
import os

for frame in glob.glob("images/pose_front/*.jpg"):
    path, basename = os.path.split(frame)
#     print(path, basename)
    result = detect(frame)
    outfile = os.path.join("results/pose_front/", basename)
    rgbimg = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    cv2.imwrite(outfile, rgbimg)
    print("wrote {}...".format(outfile))

write frames to video with ffmpeg:

In [ ]:
!ffmpeg -framerate 30 -i results/pose_front/%04d.jpg -c:v libx264 -crf 0 output.mp4

### Detect pose rear perspective

In [ ]:
import glob
import os

for frame in glob.glob("images/pose_rear/*.jpg"):
    path, basename = os.path.split(frame)
#     print(path, basename)
    result = detect(frame)
    outfile = os.path.join("results/pose_rear/", basename)
    rgbimg = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    cv2.imwrite(outfile, rgbimg)
    print("wrote {}...".format(outfile))

In [ ]:
# write frames to file with ffmpeg
!ffmpeg -framerate 30 -i results/pose_rear/%04d.jpg -c:v libx264 -crf 0 results/detected_pose_rear.mp4

another try